In [8]:
import keras
import datetime
import tensorflow                       as tf
from tensorflow.keras.callbacks         import TensorBoard
from tensorflow.keras.layers            import Input, Lambda, Conv2D,Dropout,MaxPooling2D,Conv2DTranspose,concatenate,BatchNormalization, Activation
from tensorflow.keras.models            import Model
from tensorflow.keras.optimizers        import Adam
from keras.utils                        import plot_model
from tensorflow.keras                   import layers, models
from tensorflow.keras.losses            import mae
import sys
import os
import numpy as np
import math
import random, time

import skimage                      as ski
from   skimage.filters              import threshold_otsu
from   skimage                      import io, color
from   skimage.color                import rgb2gray
from   skimage                      import filters
import cv2                          as cv
import matplotlib.pyplot            as plt 
from sklearn.model_selection import train_test_split
import gc
import glob


In [5]:
# Main directory 
mainPath = "/home/ppgi/Trabajo/Codigos_generate_data/DLCODES-VER-5"

directory= os.path.abspath(mainPath)

# Add directory to sys.path
sys.path.append(directory)

# import script where parameters are defined
import param  

In [6]:
# Vizualizancing main directories 
list_of_paths = param.list_paths 

# Defining amount of train,valid,test images   
n_train=param.n_train;    n_valid=param.n_valid;    n_test=param.n_test

# Defining a number of samples
n_sample=param.n_sample

# For resizing dimentions
h=param.img_height
w=param.img_width

#Pack them into batches (we’ll use batches of 10 images e.g.)
BATCH_SIZE=param.pack_size 

'''
Showing parameters
'''
for i,j in enumerate(list_of_paths):
    print(f'{i} => {j}')

print(f'\nTrain = {n_train}, Test = {n_test}, Valid = {n_valid}\n')
print(f'Dimensions resized h = {h}, w = {w}\n')
print(f'Pack into {BATCH_SIZE} images')  

0 => /home/ppgi/Trabajo/Codigos_generate_data/DLCODES-VER-5/Geometry
1 => /home/ppgi/Trabajo/Codigos_generate_data/DLCODES-VER-5/Magnitude
2 => /home/ppgi/Trabajo/Codigos_generate_data/DLCODES-VER-5/Pression
3 => /home/ppgi/Trabajo/Codigos_generate_data/DLCODES-VER-5/U001
4 => /home/ppgi/Trabajo/Codigos_generate_data/DLCODES-VER-5/U002

Train = 0.8, Test = 0.2, Valid = 0.0

Dimensions resized h = 128, w = 256

Pack into 100 images


In [7]:
#method for reading image    
def get_img(img_name):
    return ski.io.imread(img_name)

# method for turning to a grey or binary image 
def processing(image,option=True):
        x = get_img(image)  
        x = rgb2gray(x)       # It returns a grayscale image with floating point values in the range from 0 to 1
        x =cv.resize(x,(w,h)) # Reshape image 
    
        # Binary option otherwise gray
        if (option):
            x=ski.util.img_as_ubyte(x)  # Convert it back to the original data type and the data range back 0 to 255. 
                                        # It is often better to use image values represented by floating point values 
            best_value_threshold=np.round(filters.threshold_otsu(x)) #  Otsu’s method calculates an “optimal” threshold

            _,x= cv.threshold(x, best_value_threshold, 255, cv.THRESH_BINARY)
            x=x/255.
        x=x[:, :, np.newaxis]
        return x

In [ ]:
#  geo_train,geo_valid,geo_tests=split_data(list_of_paths[0],n_sample,n_train,n_valid,n_test) 
#  geo_array_train=arrays_img_processed(geo_train)
def  process_by_batches(list_of_path,number_batch,option=True):
    files=sorted([os.path.join(list_of_paths,fname) for fname in os.listdir(list_of_paths) if fname.endswith(".png")])
    n_files=len(files)
    for k,start in enumerate(range(0,n_files,number_batch)):
         batch=[]
         end = min(start + number_batch, n_files)
         file_batches=files[start:end]
         for img in file_batches:
             x = processing(img,option)
             batch.append(x)
         np.save(f'batch_images_{k:03d}.npy',batch,allow_pickle=True)   
         del batch
         gc.collect()
        

           
         

In [ ]:
for i, batch in enumerate(lista_de_batches):
    np.save(f'batch_{i:03d}.npy', batch)
# Busca todos los archivos .npy
lote_files = sorted(glob.glob("batch_*.npy"))

# Cargar y concatenar
lotes = [np.load(f) for f in lote_files]
todo = np.concatenate(lotes, axis=0)  # Eje 0: une verticalmente

print("Shape total:", todo.shape)  

from sklearn.model_selection import train_test_split

# Supongamos que `todo` es tu array (ej: imágenes)
X_train, X_test = train_test_split(todo, test_size=0.2, random_state=42)

lista_cargada = np.load('lista_de_arrays.npy', allow_pickle=True)

lista_cargada = lista_cargada.tolist()
